LAS file Processing Tools

# Tools for Deriving Products from LAS files
## Introduction
This jupyter notebook is designed to enable the efficient processing of LAS files. LAS files are an industry standard binary format for storing aerial LiDAR data. The python script in this notebook completes a number of processes useful for analysis. 
**Note:  As this script uses the arcpy package, it can only be run on a  machine with Arcgis Pro installed although ArcGIS Pro doesn't need to be running.**
## Workflow
The following processes are carried out in this script.
1. A Las dataset is created from the LAS files. A LAS dataset is a file storage container for LAS files used by ESRI
2. The ground surface is extracted and classified. This is the process where all the points in the cloud are processed and those deemed to form the bare earth surface are given a classification of "2" which is the designation for ground points in the LAS specification.
3. A digital Surface Model (DSM) of the pointcloud is generated. This is a raster file with the pixels representing the elevation of the non classified cloud.
4. A Digital Terrain Model is created of the ground. This is a raster file with the pixels representing the elevation of the points designated as bare earth.
5. A difference raster is created by subtracting the DTM from the DSM to show the vegetation height in the area of interest
6. If a shapefile is included containing a line feature class representing cross sections then these will be extracted and plotted in the notebook as well as being exported as a jpeg and a CSV file.

** A sample dataset containing a LAS file and a shapefile can be downloaded [here](https://drive.google.com/drive/folders/1oWOcZGH8XyItQWyAD-vEmMEGzo5CKYn_?usp=sharing)



## Cell 1: Impoting the required modules. 
Click in the cell below a click run to import the necessary python modules. When it is finished you will see a message saying "Completed".

In [4]:
import arcpy
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import os
print('Completed')

Completed


## Cell 2: Inputting the variables
The cell below requires the user to input the paths and names for various inputs and outputs. If the user is mirroring the same folder naming convention and structure as the default, then no changes are needed. (See the user manual for a full description of the default folder structure). If the user has a different folder structure, then the paths in the variables below need to be altered accordingly.

**Note that all but one of the path variables below reference a folder location. For this reason, they are referenced with a double backslash. The user will need to replicate this if copying and pasting a path to their own folder location. The variable "pointcloud_path" references the Pointcloud.las file and not the folder containing it so the path to the users own LAS file can be pasted within the quotes. For this reason, the LAS file can have any name.

##### arcpy_env_workspace
This variable creates a temporary file geodatabase called temp.gdb for storing layers etc while the tool is running. It can be deleted when the session is finished.

##### pointcloud_path
This is the path to the pointcloud LAS file.

##### raster_folder
This is the path where the raster output (DTM and DSMs) will be stored

##### ground_DSM 
This is the name of the raster representing the dround surface. The user can use whatever name they choose but the file extension should be .tif format

##### surface_model
This is the name to be given to the raster representing the DSM. Again the user can choose their own name.

The other two variables (**ground_raster_full_path** and **surface_raster_full_path**) are used internally within the code and should not be set by the user.

In [15]:
arcpy_env_workspace = "C:\\DSM_Tools\\"  # enter path and name of workspace here
pointcloud_path = r"C:\DSM_Tools\LAS_Processing_Tools\Pointcloud_Data\Pointcloud.las" #enter the path to your LAS files
las_dataset_path = "C:\\DSM_Tools\\LAS_Processing_Tools\\Pointcloud_Data\\"# enter las dataset path here
raster_folder = "C:\\DSM_Tools\\LAS_Processing_Tools\\rasters\\"  #enter path to where output rasters ar to be stored
ground_dsm = "ground_DSM.tif"
surface_model = "surface_DSM.tif"
ground_raster_full_path = os.path.join(raster_folder, ground_dsm)
surface_raster_full_path = os.path.join(raster_folder, ground_dsm)
print('Completed')

Completed


## Cell 3
This cell creates the temporary file geodatabase in the folder specified in the previous cell. If there is a geodatabase of the same name already there, it will be deleted.This cell needs no input from the user and can be run as it is.

In [18]:
#create a temporary geodatabase in workspace
if arcpy.Exists(arcpy_env_workspace +"temp.gdb"):
    arcpy.Delete_management(arcpy_env_workspace +"temp.gdb")
arcpy.management.CreateFileGDB(arcpy_env_workspace, 'temp')

<Result 'C:\\DSM_Tools\\temp.gdb'>

## Cell 4
This cell uses arcpy to create a las dataset which is a pointer to a set of related las files. The parameters that need to be set here are...

Input: "NO_RECURSION" - only las files in the input folder are added to the dataset or "RECURSION" - las files in subdirectories will be added to the dataset.

Surface Constraints: Any polygons or other feature classes to be referenced by the LAS dataset. The default is "None". See arcgis pro help for more info if feature types are to be included.

Coordinate reference system: This is the EPSG code of the CRS for the LAS files. If this is not set then the CRS will be unknown. The default used here is "27700" which is the British National Grid.

Compute Stats: This is optional and can be set to "COMPUTE_STATS" or "NO_COMPUTE_STATS"(default). 





In [31]:
#First make las dataset from pointcloud
arcpy.management.CreateLasDataset(pointcloud_path,
                                  las_dataset_path+"Pointcloud.lasd", "NO_RECURSION", None,
                                 arcpy.SpatialReference(27700), "COMPUTE_STATS")

<Result 'C:\\DSM_Tools\\LAS_Processing_Tools\\Pointcloud_Data\\Pointcloud.lasd'>

## Cell 5


In [7]:
##Classify the ground surface

arcpy.ddd.ClassifyLasGround(las_dataset_path+"Pointcloud.lasd", 
                            "AGGRESSIVE", "RECLASSIFY_GROUND", None, "COMPUTE_STATS", "DEFAULT", None, 
                            "PROCESS_EXTENT")


<Result 'C:\\DSM_Tools\\LAS_Processing_Tools\\Pointcloud_Data\\Pointcloud.lasd'>

In [8]:
 
if arcpy.Exists("Pointcloud_LasDatasetLayer"):
    arcpy.Delete_management("Pointcloud_LasDatasetLayer")
    

arcpy.management.MakeLasDatasetLayer(las_dataset_path+"Pointcloud.lasd", "Pointcloud_LasDatasetLayer", "2", None, "INCLUDE_UNFLAGGED", "INCLUDE_SYNTHETIC", "INCLUDE_KEYPOINT", "EXCLUDE_WITHHELD", None, "INCLUDE_OVERLAP")

<Result 'Pointcloud_LasDatasetLayer'>

In [13]:
if arcpy.Exists(ground_raster_full_path):
    arcpy.Delete_management(ground_raster_full_path)
arcpy.conversion.LasDatasetToRaster("Pointcloud_LasDatasetLayer", ground_raster_full_path, "ELEVATION", "BINNING AVERAGE LINEAR", "FLOAT", "CELLSIZE", 0.1, 1)


<Result 'C:\\DSM_Tools\\LAS_Processing_Tools\\rasters\\ground_DSM.tif'>